In [ ]:
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten, AveragePooling2D
from tensorflow.keras.utils import Sequence
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score
from sklearn.metrics import f1_score, accuracy_score
import plotly.figure_factory as ff
import plotly
import plotly.graph_objs as go
import pickle
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from statistics import mode, StatisticsError
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
import time

start = time.time()

In [ ]:
t0 = time.time()
bk_dataset = np.load('E:\\TwoHand_75Stack_5inputs.npz')

data = bk_dataset['data']
label = bk_dataset['label']

t1 = time.time()
print('loading : %f'%(t1-t0), ' s')

loading : 63.830064  s


In [ ]:
data.shape , label.shape

((789, 75, 224, 224, 3), (789,))

In [ ]:
label = np.array(label)

In [ ]:
# Split Data
# train 80% , test = 20%
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.2, shuffle=True, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(631, 75, 224, 224, 3) (158, 75, 224, 224, 3) (631,) (158,)


In [ ]:
x_train /= 255.0
x_test /= 255.0

In [ ]:
y_train_1h = pd.get_dummies(y_train) # 1 hot format
y_test_1h = pd.get_dummies(y_test)
print(y_test_1h.head(20))
print(y_train_1h.shape)
print(y_test_1h.shape)

    ทัณฑฆาต  ฤ  อะ  อา  อำ  อุ  อู  อ่  อ้  อ๊  อ๋  ฯ  เอ  แอ  ไม้หันอากาศ  \
0         0  0   0   0   0   0   1   0   0   0   0  0   0   0            0   
1         0  0   0   0   0   0   0   1   0   0   0  0   0   0            0   
2         1  0   0   0   0   0   0   0   0   0   0  0   0   0            0   
3         1  0   0   0   0   0   0   0   0   0   0  0   0   0            0   
4         1  0   0   0   0   0   0   0   0   0   0  0   0   0            0   
5         1  0   0   0   0   0   0   0   0   0   0  0   0   0            0   
6         0  0   0   0   0   0   0   0   0   0   0  1   0   0            0   
7         0  0   0   1   0   0   0   0   0   0   0  0   0   0            0   
8         0  0   1   0   0   0   0   0   0   0   0  0   0   0            0   
9         0  0   0   0   0   0   0   0   0   1   0  0   0   0            0   
10        0  0   0   0   0   0   0   0   0   0   0  1   0   0            0   
11        1  0   0   0   0   0   0   0   0   0   0  0   0   0   

In [ ]:
y_train_1h.shape[1]

16

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten, AveragePooling2D
from tensorflow.keras.utils import Sequence
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score
from sklearn.metrics import f1_score, accuracy_score
import plotly.figure_factory as ff
import plotly
import plotly.graph_objs as go
import pickle
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from statistics import mode, StatisticsError
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import time

In [ ]:
def LSTM_model_TwoHand():
    model = Sequential()

    model.add(LSTM(units = 128, input_shape=(75, 1), activation= 'tanh', return_sequences= True))
    model.add(LSTM(units = 64, activation='tanh', return_sequences=True))
    model.add(LSTM(units= 32, activation= 'tanh'))

    # model.add(Flatten())
    model.add(Dense(units = 64, activation = 'tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 32, activation = 'tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units= y_train_1h.shape[1], activation= 'softmax'))
    model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = LSTM_model_TwoHand()
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75, 128)           66560     
                                                                 
 lstm_1 (LSTM)               (None, 75, 64)            49408     
                                                                 
 lstm_2 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
Total params: 133,104
Trainable params: 133,104
Non-trai

In [ ]:
# DataGenerator
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(x_train, y_train_1h, 8)
test_gen = DataGenerator(x_test, y_test_1h, 8)

In [ ]:
es = EarlyStopping(monitor='val_loss', verbose=1, patience=5) #
mc = ModelCheckpoint('test_bk_TwoHand.h5', monitor='val_accuracy', verbose=1, save_best_only=True) #
history = model.fit(train_gen,epochs=50,validation_data=test_gen, verbose=1, validation_split=0.2, callbacks=[es, mc])
# history = model.fit(x_train,y_train_1h,epochs=25,batch_size=2,verbose=1,validation_split=0.2)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class '__main__.DataGenerator'>]